In [ ]:
import tkinter as tk
from tkinter import messagebox
import requests
import os
from dotenv import load_dotenv
import logging
#import webbrowser
#webbrowser.open("https://www.openweathermap.org/")


# Load API key from .env
load_dotenv()
API_KEY ='f0dbfad25cd12c693ac31e65226c718a'

# Log file setup
LOG_FILE = "app.log"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

# ---------------------- Helper Functions ------------------------

def get_weather(city):
    try:
        params = {
            'q': city,
            'appid': API_KEY,
            'units': 'metric'
        }
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        if response.status_code == 200:
            logging.info(f"Fetched weather for {city}")
            return {
                'city': data['name'],
                'temp': data['main']['temp'],
                'humidity': data['main']['humidity'],
                'wind': data['wind']['speed'],
                'lat': data['coord']['lat'],
                'lon': data['coord']['lon']
            }
        else:
            error_msg = data.get("message", "Unknown error.")
            logging.error(f"Weather error: {error_msg}")
            return {"error": error_msg}
    except Exception as e:
        logging.error(f"Weather error: {e}")
        return {"error": str(e)}

def calculate(a, b, operation):
    try:
        a = float(a)
        b = float(b)
        if operation == '+':
            result = a + b
        elif operation == '-':
            result = a - b
        elif operation == '*':
            result = a * b
        elif operation == '/':
            if b == 0:
                return "Cannot divide by zero."
            result = a / b
        else:
            return "Invalid operation."

        op_str = f"{a} {operation} {b} = {result}"
        logging.info(f"Calculation: {op_str}")
        return op_str
    except ValueError:
        return "Invalid input."

# ---------------------- UI Windows ------------------------

def open_weather_window():
    win = tk.Toplevel()
    win.title("Weather App")
    win.geometry("400x300")

    tk.Label(win, text="Enter City Name:").pack(pady=10)
    city_entry = tk.Entry(win)
    city_entry.pack(pady=5)

    result_label = tk.Label(win, text="", justify="left")
    result_label.pack(pady=10)

    def fetch_weather():
        city = city_entry.get().strip()
        if not city:
            messagebox.showwarning("Input Error", "Please enter a city name.")
            return
        info = get_weather(city)
        if 'error' in info:
            result_label.config(text=f"Error: {info['error']}")
        else:
            result_label.config(text=(
                f"City: {info['city']}\n"
                f"Temperature: {info['temp']}°C\n"
                f"Humidity: {info['humidity']}%\n"
                f"Wind Speed: {info['wind']} m/s\n"
                f"Coordinates: ({info['lat']}, {info['lon']})"
            ))

    tk.Button(win, text="Get Weather", command=fetch_weather).pack(pady=5)

def open_calculator_window():
    win = tk.Toplevel()
    win.title("Calculator")
    win.geometry("400x300")

    tk.Label(win, text="First Number:").pack()
    entry1 = tk.Entry(win)
    entry1.pack()

    tk.Label(win, text="Operation (+, -, *, /):").pack()
    entry_op = tk.Entry(win)
    entry_op.pack()

    tk.Label(win, text="Second Number:").pack()
    entry2 = tk.Entry(win)
    entry2.pack()

    result_label = tk.Label(win, text="", fg="blue")
    result_label.pack(pady=10)

    def calculate_result():
        a = entry1.get()
        op = entry_op.get()
        b = entry2.get()
        result = calculate(a, b, op)
        result_label.config(text=result)

    tk.Button(win, text="Calculate", command=calculate_result).pack(pady=5)

def open_history_window():
    win = tk.Toplevel()
    win.title("History Log")
    win.geometry("500x400")

    tk.Label(win, text="📜 Operation History", font=("Arial", 14)).pack(pady=10)

    text_area = tk.Text(win, wrap='word', font=("Courier", 10))
    text_area.pack(expand=True, fill='both', padx=10, pady=10)

    try:
        with open(LOG_FILE, 'r') as file:
            content = file.read()
            text_area.insert('1.0', content)
    except FileNotFoundError:
        text_area.insert('1.0', "No history found.")

# ---------------------- Main Menu ------------------------

def main():
    root = tk.Tk()
    root.title("Weather & Calculator App")
    root.geometry("300x300")

    tk.Label(root, text="Main Menu", font=("Arial", 16)).pack(pady=10)

    tk.Button(root, text="🌤️ Weather App", command=open_weather_window, width=20).pack(pady=10)
    tk.Button(root, text="🧮 Calculator", command=open_calculator_window, width=20).pack(pady=10)
    tk.Button(root, text="📖 View History", command=open_history_window, width=20).pack(pady=10)
    tk.Button(root, text="❌ Exit", command=root.destroy, width=20).pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    main()
